# Calculate Raster Analysis

### This notebook calculates the following items referenced in the paper:

<b>Disadvantaged Community (DAC) intersections </b>
* Amount of land (km-squared) used by new infrastructure under each scenario that intersects with federally-ideantified DACs
* Amount of land (km-squared) in each state used by new infrastructure under each scenario that intersects with federally-ideantified DACs
* Amount of land (km-squared) used by new infrastructure under each scenario that intersects with federally-ideantified DACs by technology
* Amount of land (km-squared) in each state used by new infrastructure under each scenario that intersects with federally-ideantified DACs by technology type

<b>Important Farmland</b>
* Amount of land (km-squared) used by new infrastructure under each scenario that intersects with USDA-defined Important Farmland
* Amount of land (km-squared) in each state used by new infrastructure under each scenario that intersects with USDA-defined Important Farmland
* Amount of land (km-squared) used by new infrastructure under each scenario that intersects with USDA-defined Important Farmland by technology
* Amount of land (km-squared) in each state used by new infrastructure under each scenario that intersects with USDA-defined Important Farmland by technology type

<b>Proximity to Environmental Areas</b>
* Fraction of new infrastructure by technology type under each scenario that is within 1, 5, and 10-km of environmental/protected areas
* Fraction of new infrastructure by technology type under each scenario that is within 1, 5, and 10-km of environmental/protected areas by state


## Raster Data

Most of the raster data used for this analysis is from the GRIDCERF geospatial database v1.0.0, accessible at https://zenodo.org/records/6601790. Additional raster data from GRICERF v2.0.0 was used to supplement the dataset, accessible at: https://doi.org/10.57931/2281697.To prepare data for analysis, all raster data is clipped to the western interconnection boundary extent.

### Environmental Proximity Raster Data
The `west_analysis_combined_environment.tif` raster used for the environmental proximity analysis is a combination of the following raster files from the GRIDCERF database v1.0.0 and supplemented with data files from GRIDCERF v2.0.0:

| Suitability Layer | GRIDCERF Raster Name | GRIDCERF version |
|-------------------|----------------------|----------------------|
|Agricultural Research Service Lands |gridcerf_ars_lands_2020_conus.tif| v1.0.0 |
|Bureau of Land Management (BLM) National Landscape Conservation System (NLCS) - National Monuments |gridcerf_blm_nlcs_national_monument_2021_conus.tif | v1.0.0 |
|BLM NLCS - Outstanding Natural Areas|gridcerf_blm_nlcs_outstanding_natural_areas_2017_conus.tif|v1.0.0 |
|BLM NLCS - Trails Historic West|gridcerf_blm_nlcs_trails_historic_west_buff_1km_2019_conus.tif|v1.0.0 |
|BLM NLCS System - Trails Scenic East|gridcerf_blm_nlcs_trails_scenic_east_buff_1km_2019_conus.tif|v1.0.0 |
|BLM NLCS System – Wilderness|gridcerf_blm_nlcs_wilderness_2021_conus.tif|v1.0.0 |
|BLM NLCS - Wilderness Study Areas|gridcerf_blm_nlcs_wilderness_study_areas_2021_conus.tif|v1.0.0 |
|BLM NLCS - Scenic Rivers|gridcerf_blm_scenic_rivers_1km_2009_conus.tif|v1.0.0 |
|National Park Service (NPS) Class 1 airsheds|gridcerf_class1_airsheds_2015_conus.tif|v1.0.0 |
|BLM NLCS National Conservation Areas|gridcerf_cons_monu_desig_2021_conus.tif|v1.0.0 |
|U.S. Fish and Wildlife Service (USFWS) - Critical Habitat|gridcerf_fws_critical_habitat_2019_conus.tif|v1.0.0 |
|USFWS - Land Interests|gridcerf_fws_land_interests_2019_conus.tif|v1.0.0 |
|USFWS - Lands|gridcerf_fws_lands_2021_conus.tif|v1.0.0 |
|USFWS - National Wildlife Refuges|gridcerf_fws_national_wildlife_refuges_2019_conus.tif|v1.0.0 |
|USFWS - Special Designation|gridcerf_fws_special_designation_2019_conus.tif|v1.0.0 |
|National Land Cover Dataset (NLCD) Wetlands|gridcerf_nlcd_wetlands_1km_2019_conus.tif|v1.0.0 |
|NPS Administrative Boundaries|gridcerf_nps_administrative_boundaries_2020_conus.tif|v1.0.0 |
|NPS Lands|gridcerf_nps_lands_2019_conus.tif|v1.0.0 |
|BLM NLCS - Wild & Scenic Rivers|gridcerf_nwrs_buff_1km_2021_conus.tif|v1.0.0 |
|U.S. Forest Service (USFS) Administrative Boundaries|gridcerf_usfs_administrative_boundaries_2021_conus.tif|v1.0.0 |
|USFS lands|gridcerf_usfs_lands_2021_conus.tif|v1.0.0 |
|U.S. Geological Survey (USGS) National Wilderness Lands|gridcerf_wilderness_lands_2021_conus.tif|v1.0.0 |
|USGS Protected Areas of the U.S - Class 1&2|gridcerf_usgs_padus_class_1_to_2_2018_conus.tif|v1.0.0 |
|U.S. State Protected Lands|gridcerf_wdpa_state_protected_lands_2021_conus.tif|v1.0.0 |
|Nature Conservancy lands|gridcerf_wdpa_tnc_managed_lands_2016_conus.tif|v1.0.0 |
|Bureau of ocean energy management offshore areas withdrawn from leasing |gridcerf_offshore_areas_withdrawn_from_leasing.tif|v2.0.0 |
|National Oceanic Atmospheric Administration, Marine Sanctuaries|gridcerf_noaa_marine_sanctuaries.tif|v2.0.0 |
|NOAA Habitats of Particular Concern|gridcerf_noaa_habitats_of_particular_concern.tif|v2.0.0 |

The `west_analysis_combined_environment_1buff.tif`, `west_analysis_combined_environment_5buff.tif`, `west_analysis_combined_environment_10buff.tif` rasters are the `west_analysis_combined_environment.tif` layer with 1, 5, and 10km buffer added, respectively.

### Disadvantaged Communities Raster Data

The disadvantaged community raster data used in the analysis is from GRIDCERF v2.0.0. the `west_usceq_combined_vulnerable_communities.tif` file is a combination of each of the individual DAC classifications shown below:

| Suitability Layer | GRIDCERF Raster Name | GRIDCERF version |
|-------------------|----------------------|----------------------|
|US CEQ CEJST Climate Disadvantaged Communities |gridcerf_usceq_cejst_exclude_climate_dacs.tif|v2.0.0 |
|US CEQ CEJST Energy Disadvantaged Communities |gridcerf_usceq_cejst_exclude_climate_dacs.tif|v2.0.0 |
|US CEQ CEJST Health Disadvantaged Communities |gridcerf_usceq_cejst_exclude_climate_dacs.tif|v2.0.0 |
|US CEQ CEJST Housing Disadvantaged Communities |gridcerf_usceq_cejst_exclude_climate_dacs.tif|v2.0.0 |
|US CEQ CEJST Pollution Disadvantaged Communities |gridcerf_usceq_cejst_exclude_climate_dacs.tif|v2.0.0 |
|US CEQ CEJST Transportation Disadvantaged Communities |gridcerf_usceq_cejst_exclude_climate_dacs.tif|v2.0.0 |
|US CEQ CEJST Water Disadvantaged Communities |gridcerf_usceq_cejst_exclude_climate_dacs.tif|v2.0.0 |

### Important Farmland Raster Data

The important farmland raster data used in the analysis is from GRIDCERF v2.0.0. the `west_usda_combined_important_farmland.tif` file is a combination of each of the individual farmland classifications shown below:

| Suitability Layer | GRIDCERF Raster Name | GRIDCERF version |
|-------------------|----------------------|----------------------|
|US Department of Agriculture, Farmland of Local Importance |gridcerf_usda_nrsc_farmland_of_local_importance.tif|v2.0.0 |
|US Department of Agriculture, Farmland of State Importance |gridcerf_usda_nrsc_farmland_of_state_importance.tif|v2.0.0 |
|US Department of Agriculture, Farmland of Unique Importance  |gridcerf_usda_nrsc_farmland_of_unique_importance.tif|v2.0.0 |
|US Department of Agriculture, Prime Farmland |gridcerf_usda_nrsc_prime_farmland_classification.tif|v2.0.0 |


<h2> Methodology</h2>

To identify how much land used by new power plant infrastructure intersects with areas of interest, we go through the following steps:
<ol>
    <li> Select power plant sitings for the given scenario that correspond to new infrastructure only. </li>
    <li> Extract x and y coordinates for each of the power plants. </li>
    <li> For each set of coordinates, determine if the value for the associated 1km grid cell in the raster file of interest is equal to 1 (indicates that that grid cell is positive for the given indicator (e.g., is important farmland), or 0. </li>
    <li> Sum the number of power plants that intersect with raster values equal to 1</li>
    
</ol>

## imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from shapely import Point

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap

import rasterio
from rasterio.plot import show

import seaborn as sns

import os
from pathlib import Path

### Settings

In [2]:
# list of scenarios for analysis
scenario_list = ['net_zero_ira_ccs_climate', 'business_as_usual_ira_ccs_climate']

# list of years for analysis
year = 2050

color_map_dict={
    'Natural Gas':'#bdb59d',
    'Coal': '#75736d',
    'Solar PV': '#F7B615',
    'Solar CSP': '#f1781b',
    'Wind': '#95C060', 
    'Nuclear': '#6D547C', 
    'Offshore Wind':'#2B7F9E', 
    'Biomass': '#713275',
    'Geothermal': '#BB3B1C', 
    'Oil': '#BB3B1C',
    'Hydro': '#14345e'}

tech_rename_dict = {
    'gridcerf_biomass_conventional_no-ccs_recirculating': 'Biomass (recirculating cooling)',
    'gridcerf_biomass_conventional_no-ccs_dry' : 'Biomass (dry cooling)',
    'gridcerf_biomass_conventional_ccs_dry':'Biomass CCS (dry cooling)',
    'gridcerf_biomass_conventional_ccs_recirculating': 'Biomass CCS (recirculating cooling)',

    'gridcerf_biomass_igcc_ccs_dry':'Biomass IGCC CCS (dry cooling)',
    'gridcerf_biomass_igcc_ccs_recirculating':'Biomass IGCC CCS (recirculating cooling)',
    'gridcerf_biomass_igcc_ccs_recirculating-seawater':'Biomass IGCC CCS (seawater cooling)',
    'gridcerf_coal_conventional_ccs_dry':'Coal CCS (dry cooling)',
    'gridcerf_coal_conventional_ccs_recirculating':'Coal CCS (recirculating cooling)',
    
    'gridcerf_gas_cc_no-ccs_recirculating': 'Natural Gas CC (recirculating cooling)',
    'gridcerf_gas_cc_no-ccs_dry': 'Natural Gas CC (dry cooling)',
    'gridcerf_gas_cc_no-ccs_pond':'Natural Gas CC (pond cooling)',
    
    'gridcerf_gas_cc_ccs_dry':'Natural Gas CC CCS (dry cooling)',
    'gridcerf_gas_cc_ccs_recirculating': 'Natural Gas CC CCS (recirculating cooling)',
    'gridcerf_gas_cc_ccs_recirculating-seawater': 'Natural Gas CC CCS (seawater cooling)',

    'gridcerf_solar_csp_centralized_dry-hybrid': 'Solar CSP (dry-hybrid cooling)',
    'gridcerf_solar_csp_centralized_recirculating': 'Solar CSP (recirculating cooling)',
    
    'gridcerf_solar_pv_centralized_no-cooling': 'Solar PV',
    'gridcerf_wind_onshore_hubheight100_no-cooling': 'Wind',
    'gridcerf_wind_onshore_hubheight120_no-cooling': 'Wind',
    'gridcerf_wind_onshore_hubheight140_no-cooling':'Wind',
    'gridcerf_wind_offshore_hubheight140_no-cooling': 'Offshore Wind',
    'gridcerf_wind_offshore_hubheight160_no-cooling':'Offshore Wind',
    
    'gridcerf_gas_cc_no-ccs_recirculating-seawater': 'Natural Gas CC (seawater cooling)',
    'gridcerf_geothermal_centralized_enhanced_recirculating' : 'Geothermal (recirculating cooling)',
    'gridcerf_nuclear_gen3_smr_recirculating': 'Nuclear SMR (recirculating cooling)'}


### Collect Data Paths

In [4]:
# data dir
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'input_data')

# output data dir
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'output_data')

# output figure dir
fig_dir = os.path.join(os.path.dirname(os.getcwd()), 'figures')

# western interconnection shapefile
wecc_shp_path =  os.path.join(data_dir, "wecc_boundary_shp", "wecc_boundary.shp")

# western interconnection transmission shapefile
transmission_path = os.path.join(data_dir, 'transmission_shp', 'wecc_transmission_250kv.shp')

# infrastructure siting output csv
infrastucture_path = os.path.join(data_dir, 'infrastructure_data_csv', f'infrastructure_data_{year}.csv')


# land area files
land_area_raster_dir = os.path.join(data_dir, "west_raster_data", 'land_area')

# all area files
all_area_raster_dir = os.path.join(data_dir, "west_raster_data", 'all_area')

# # raster data directory
# raster_dir = os.path.join(data_dir, 'raster_data')

# # environmental raster dir
# env_dir = os.path.join(raster_dir, 'environmental')

# # disadvantaged community raster dir
# dac_dir = os.path.join(raster_dir, 'disadvantaged_communities')

# # environmental raster dir
# farm_dir = os.path.join(raster_dir, 'important_farmland')

# data output paths

# dac analysis files
cg_dac_path = os.path.join(output_dir,  f'clean_grid_dac_analysis_{year}.csv')
bau_dac_path = os.path.join(output_dir, f'bau_dac_analysis_{year}.csv')
diff_dac_path = os.path.join(output_dir, f'difference_dac_analysis_{year}.csv')

# farmland dir
cg_farm_path = os.path.join(output_dir, f'clean_grid_farm_analysis_{year}.csv')
bau_farm_path = os.path.join(output_dir, f'bau_farm_analysis_{year}.csv')
diff_farm_path = os.path.join(output_dir, f'difference_farm_analysis_{year}.csv')

# environmental dir
cg_env_path = os.path.join(output_dir, f'clean_grid_env_analysis_{year}.csv')
bau_env_path = os.path.join(output_dir, f'bau_env_analysis_{year}.csv')
diff_env_path = os.path.join(output_dir, f'difference_env_analysis_{year}.csv')


### Functions

In [5]:
def results_to_geodataframe(df, crs = "ESRI:102003"):
    """ 
    Takes a pandas DataFrame with x and y coordinates as input and 
    converts to a GeoPandas GeodataFrame. Coordinates in DataFrame are expected to
    follow the ESRI:102003 albers equal area conic coordinate referece system.
    x-coordinate column should be called 'xcoord', y-coordinate column should be
    called 'ycoord'

    :param df:        input Pandas DataFrame with x/y coordinates
    :type df:         Pandas DataFrame
    
    :param crs:       Coordinate reference system to use for GeoDataFrame
    :type crs:        str
    
    """
    
    # create geometry column from coordinate fields
    geometry = [Point(xy) for xy in zip(df['xcoord'], df['ycoord'])]
    
    gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return gdf

def extract_coords(df):
    """
    Collects a list of x/y coordinates from a dataframe as a list of tuples.
    """

    # extract x,y coords to list of tuples
    coords = [(x,y) for x, y in zip(df['xcoord'], df['ycoord'])]
    
    return coords


def analyze_intersections(df, raster_list, layer_name_dict):
    """
    Reads in a pandas DataFrame of power plant sitings and calculates the number of sitings that intersect
    with the provided rasters given in the raster_list praameter.
    """

    # rename technologies to simplified form
    df['tech_name'] = df['tech_name'].map(tech_rename_dict)
    
    # dictionary to store outputs in
    d = {'state': [], 'tech_name': [], 'layer': [], 'total_plants':[], 'intersection': [], 'valid': []}

    for state in list(df.region_name.unique()):
        
        state_df = df[df.region_name == state]
    
        for tech in list(state_df.tech_name.unique()):
            
            total_plants = 0
    
            n_conflict_plants = 0
            
            # reduce dataframe to technology type
            df_tech = state_df[state_df.tech_name == tech]

            # extract the coordinates from the dataframe
            coords = extract_coords(df_tech)

            # total number of power plants for the CONUS
            total_plants = len(coords)

            # update suitability for each power plant once it is within a grid cell that is unsuitable
            for raster in raster_list:

                with rasterio.open(raster) as src:

                    # extract the suitability value (0 == suitable; 1 == unsuitable) for each power plant
                    extract_suitability_at_points = [int(i[0]) for i in src.sample(coords)]
    
                    # get the count of unsuitable designations
                    n_conflict_plants = len([i for i in extract_suitability_at_points if i == 1])
    
                    # append the results to the output dictionary
                    d['state'].append(state)
                    d['tech_name'].append(tech)
                    d['layer'].append(os.path.basename(raster.lower()))
                    d['total_plants'].append(total_plants)
                    d['intersection'].append(n_conflict_plants)
                    d['valid'].append(total_plants - n_conflict_plants)

    # convert the output to a pandas data frame
    layer_specific_diagnostics = pd.DataFrame(d)

    layer_specific_diagnostics['fraction'] = round(layer_specific_diagnostics['intersection'] / layer_specific_diagnostics['total_plants'], 2)

    layer_sort_list=[]
    for layer in layer_name_dict:
        layer_sort_list.append(layer)

    layer_specific_diagnostics.layer = pd.Categorical(layer_specific_diagnostics.layer, categories=layer_sort_list)
    layer_specific_diagnostics=layer_specific_diagnostics.sort_values('layer')
    
    return layer_specific_diagnostics.sort_values(by='tech_name')


def calculate_intersection_difference(clean_grid_analysis_df, bau_analysis_df):

    # rename intersection columns to scenario name
    clean_grid_analysis_df = clean_grid_analysis_df.rename(columns={'intersection':'clean_grid'})[['state', 'tech_name', 
                                                                                                   'layer', 'total_plants', 'clean_grid']]                                                                                           
    bau_analysis_df = bau_analysis_df.rename(columns={'intersection':'bau'})[['state', 'tech_name', 
                                                                              'layer','total_plants', 'bau']]
    
    # combine dataframes
    combined_analysis_df = pd.merge(clean_grid_analysis_df, bau_analysis_df, how='left', on=['state', 'tech_name', 'layer'])

    # fill nan values with 0
    combined_analysis_df['bau'].fillna(0, inplace=True)
    combined_analysis_df['clean_grid'].fillna(0, inplace=True)

    # calculate the difference in land
    combined_analysis_df['intersection'] = combined_analysis_df['clean_grid'] - combined_analysis_df['bau']

    combined_analysis_df['intersection'] = combined_analysis_df['intersection'].astype(int)

    return combined_analysis_df


### Collect Infrastructure Data

In [6]:
# collect infrastructure data
df = pd.read_csv(infrastucture_path)

# reduce to year of interest
df = df[df.sited_year <= year]

# Scenario Analysis

#### Prepare Power Plant Sitings

In [7]:
# create a dataframe of cerf power plant sitings for the clean grid scenario
df_clean_grid = df[(df.scenario == 'net_zero_ira_ccs_climate') & (df.cerf_sited == 1)].copy()

# create a dataframe of cerf power plant sitings for the business-as-usual scenario
df_bau = df[(df.scenario == 'business_as_usual_ira_ccs_climate') & (df.cerf_sited == 1)].copy()

In [18]:
# dictionary of DAC layers with their simple names
dac_dict = {'gridcerf_usceq_cejst_exclude_all_dacs.tif': 'DACs',
            'gridcerf_usceq_cejst_exclude_energy_dacs.tif': 'Energy DACs',
            'gridcerf_usceq_cejst_exclude_transportation_dacs.tif': 'Transportation DACs',
            'gridcerf_usceq_cejst_exclude_housing_dacs.tif':'Housing DACs',
            'gridcerf_usceq_cejst_exclude_pollution_dacs.tif':'Pollution DACs',
            'gridcerf_usceq_cejst_exclude_health_dacs.tif':'Health DACs',
            'gridcerf_usceq_cejst_exclude_climate_dacs.tif':'Climate DACs',
            'gridcerf_usceq_cejst_exclude_workforce_dacs.tif':'Workforce DACs',
            'gridcerf_usceq_cejst_exclude_water_dacs.tif':'Water DACs'}

# dictionary of important farmland layers with their simple names
farm_dict = {'gridcerf_usda_combined_important_farmland.tif': 'Important Farmland',
            'gridcerf_usda_nrsc_prime_farmland_classification.tif': 'Prime Farmland',
            'gridcerf_usda_nrsc_farmland_of_state_importance.tif': 'Farmland of State Importance',
            'gridcerf_usda_nrsc_farmland_of_local_importance.tif':'Farmland of Local Importance',
            'gridcerf_usda_nrsc_farmland_of_unique_importance.tif':'Farmland of Unique Importance'}

# dictionary of environmental layers with their simple names
env_dict = {'gridcerf_combined_environment.tif': 'Natural Areas',
           'gridcerf_combined_environment_1buff.tif': 'Within 1 km of Natural Area',
           'gridcerf_combined_environment_5buff.tif': 'Within 5 km of Natural Area',
           'gridcerf_combined_environment_10buff.tif': 'Within 10 km of Natural Area',
           }

## Disadvantaged Communities

<b> Scientific Question Addressed </b>: Under decarbonization and business-as-usual scenarios, how much land required for new power plant infrastructure intersects with federally-identified disadvantaged communities?

In [9]:
# create a list of paths to the DAC layers
dac_raster_list = []
for layer in dac_dict:
    layer_name = dac_dict[layer]
    layer_path = os.path.join(all_area_raster_dir, layer)
    dac_raster_list.append(layer_path)

#### Clean Grid - New Power Plant Sitings

In [10]:
# settings
input_df = df_clean_grid.copy()
raster_list = dac_raster_list
layer_dict = dac_dict
analysis_type = 'DAC Layers'
scenario = f'Clean Grid by {year}'
output_file = cg_dac_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for DAC Layers & Clean Grid by 2050...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/clean_grid_dac_analysis_2050.csv
Done


#### Business-as-usual - New Power Plant Sitings

In [11]:
# settings
input_df = df_bau.copy()
raster_list = dac_raster_list
layer_dict = dac_dict
analysis_type = 'DAC Layers'
scenario = f'Business-as-usual by {year}'
output_file = bau_dac_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for DAC Layers & Business-as-usual by 2050...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/bau_dac_analysis_2050.csv
Done


______________

## Important Farmland

<b> Scientific Question Addressed </b>: Under decarbonization and business-as-usual scenarios, how much land required for new power plant infrastructure intersects with important farmland?

In [19]:
# create a list of paths to the farmland layers
farm_raster_list = []
for layer in farm_dict:
    layer_name = farm_dict[layer]
    layer_path = os.path.join(all_area_raster_dir, layer)
    farm_raster_list.append(layer_path)

#### **Clean Grid - New Power Plant Sitings**

In [20]:
# settings
input_df = df_clean_grid.copy()
raster_list = farm_raster_list
layer_dict = farm_dict
analysis_type = 'Important Farmland Layers'
scenario = f'Clean Grid by {year}'
output_file = cg_farm_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the farmland layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for Important Farmland Layers & Clean Grid by 2050...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/clean_grid_farm_analysis_2050.csv
Done


#### **Business-as-usual - New Power Plant Sitings**

In [21]:
# settings
input_df = df_bau.copy()
raster_list = farm_raster_list
layer_dict = farm_dict
analysis_type = 'Important Farmland Layers'
scenario = f'Business-as-usual by {year}'
output_file = bau_farm_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for Important Farmland Layers & Business-as-usual by 2050...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/bau_farm_analysis_2050.csv
Done


_________

## Natural Area Proximity

<b> Scientific Question Addressed </b>: Under decarbonization and business-as-usual scenarios, how much land required for new power plant infrastructure intersects with areas in close proximity to natural areas?

In [22]:
# create a list of paths to the DAC layers
env_raster_list = []
for layer in env_dict:
    layer_name = env_dict[layer]
    layer_path = os.path.join(all_area_raster_dir, layer)
    env_raster_list.append(layer_path)

#### Clean Grid - New Power Plant Sitings

In [23]:
# settings
input_df = df_clean_grid.copy()
raster_list = env_raster_list
layer_dict = env_dict
analysis_type = 'Natural Area Proximity'
scenario = f'Clean Grid by {year}'
output_file = cg_env_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for Natural Area Proximity & Clean Grid by 2050...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/clean_grid_env_analysis_2050.csv
Done


#### **Business-as-usual - New Power Plant Sitings**

In [24]:
# settings
input_df = df_bau.copy()
raster_list = env_raster_list
layer_dict = env_dict
analysis_type = 'Natural Area Proximity'
scenario = f'Business-as-usual by {year}'
output_file = bau_env_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the DAC layers
analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for Natural Area Proximity & Business-as-usual by 2050...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/bau_env_analysis_2050.csv
Done


___________ 

# Difference Between Scenarios Analysis
This section calculates the additional land intersections between the Clean Grid and Business-as-usual scenario.

<b>Scientific Question:</b> How much <i>additional</i> land in areas of interest will achieving a decarbonized grid take? 

## Disadvantaged Communities

<b> Scientific Question Addressed </b>: How much more land required for new power plant infrastructure intersects with federall-identified disadvantaged communities in a decarbonized grid compared to a business-as-usual scenario?

In [25]:
# clean grid settings
input_df = df_clean_grid.copy()
raster_list = dac_raster_list
layer_dict = dac_dict
analysis_type = 'Disadvantaged Communities'
scenario = f'Clean Grid by {year}'

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
clean_grid_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

# bau settings
input_df = df_bau.copy()
raster_list = dac_raster_list
layer_dict = dac_dict
analysis_type = 'Disadvantaged Communities'
scenario = f'Business-as-usual by {year}'

output_file = diff_dac_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
bau_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Calculating difference in power plant intersections for {analysis_type}...')
# calculate the intersection difference between scenarios
analysis_df = calculate_intersection_difference(clean_grid_analysis_df=clean_grid_analysis_df, bau_analysis_df=bau_analysis_df)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for Disadvantaged Communities & Clean Grid by 2050...
Calculating power plant intersections for Disadvantaged Communities & Business-as-usual by 2050...
Calculating difference in power plant intersections for Disadvantaged Communities...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/difference_dac_analysis_2050.csv
Done


_______

## Important Farmland

<b> Scientific Question Addressed </b>: How much more land required for new power plant infrastructure intersects with important farmland in a decarbonized grid compared to a business-as-usual scenario?

In [26]:
# clean grid settings
input_df = df_clean_grid.copy()
raster_list = farm_raster_list
layer_dict = farm_dict
analysis_type = 'Important Farmland'
scenario = f'Clean Grid by {year}'

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
clean_grid_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

# bau settings
input_df = df_bau.copy()
raster_list = farm_raster_list
layer_dict = farm_dict
analysis_type = 'Important Farmland'
scenario = f'Business-as-usual by {year}'

output_file = diff_farm_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
bau_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Calculating difference in power plant intersections for {analysis_type}...')
# calculate the intersection difference between scenarios
analysis_df = calculate_intersection_difference(clean_grid_analysis_df=clean_grid_analysis_df, bau_analysis_df=bau_analysis_df)

print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for Important Farmland & Clean Grid by 2050...
Calculating power plant intersections for Important Farmland & Business-as-usual by 2050...
Calculating difference in power plant intersections for Important Farmland...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/difference_farm_analysis_2050.csv
Done


__________

## Natural Area Proximity

<b> Scientific Question Addressed </b>: How much more land required for new power plant infrastructure intersects with areas in close proximity to natural areas in a decarbonized grid compared to a business-as-usual scenario?

In [27]:
# clean grid settings
input_df = df_clean_grid.copy()
raster_list = env_raster_list
layer_dict = env_dict
analysis_type = 'Natural Area Proximity'
scenario = f'Clean Grid by {year}'

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
clean_grid_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

# bau settings
input_df = df_bau.copy()
raster_list = env_raster_list
layer_dict = env_dict
analysis_type = 'Natural Area Proximity'
scenario = f'Business-as-usual by {year}'

output_file = diff_env_path

print(f'Calculating power plant intersections for {analysis_type} & {scenario}...')
# conduct an analysis of intersections with each of the layers
bau_analysis_df = analyze_intersections(df=input_df, raster_list=raster_list, layer_name_dict=layer_dict)

print(f'Calculating difference in power plant intersections for {analysis_type}...')
# calculate the intersection difference between scenarios
analysis_df = calculate_intersection_difference(clean_grid_analysis_df=clean_grid_analysis_df, bau_analysis_df=bau_analysis_df)


print(f'Saving output to {output_file}')
analysis_df.to_csv(output_file, index=False)
print('Done')

Calculating power plant intersections for Natural Area Proximity & Clean Grid by 2050...
Calculating power plant intersections for Natural Area Proximity & Business-as-usual by 2050...
Calculating difference in power plant intersections for Natural Area Proximity...
Saving output to /Users/mong275/repos/mongird-etal_2024_tbd/data/output_data/difference_env_analysis_2050.csv
Done
